In [6]:
from IO.config import get_usr_config
from IO.result import DetectionResult
from core import detector
from core import freq_transform
from core import signal_generator
from core import utils
from core.utils import MICROSECONDS_IN_SECONDS
from IO import config

import matplotlib.pyplot as plt
import numpy as np
import torch

from datetime import datetime

In [7]:
class NoiseModelData:
    def __init__(
        self, 
        dim,
        Bx, Bw, Ba,
        noise_level_db,
        model_coeff_generator,
        
        deltax, deltaw, deltay,
        var_weight,
        noise_level_linear,
        
        qoutput,
        noise_to_out,
        qinput_to_out,
        qweight_to_out_by_signal,
        qweight_to_out_by_noise
    ):
        self.dim = dim
        self.Bx = Bx
        self.Bw = Bw
        self.Ba = Ba
        self.noise_level_db = noise_level_db
        self.model_coeff_generator = model_coeff_generator
        self.deltax = delta
        self.deltaw = deltaw
        self.deltay = deltay
        self.var_weight = var_weight
        self.noise_level_linear = noise_level_linear
        self.noise_to_out = noise_to_out
        self.qoutput = qoutput
        self.noise_to_out = noise_to_out,
        self.qinput_to_out = qinput_to_out
        self.qweight_to_out_by_signal = qweight_to_out_by_signal
        self.qweight_to_out_by_noise = qweight_to_out_by_noise

In [8]:
def unisign_quant(data, n_bits, clip, quant_flag=False):
    if not quant_flag:
        return data
    
    data = torch.Tensor(data)
    w_c = data.clamp(-clip, clip)
    b = torch.pow(torch.tensor(2.0), 1 - n_bits)
    w_q = clip * torch.min(b * torch.round(w_c / (b * clip)), 1 - b)

    return w_q.numpy()


In [12]:
weight_quant = False
acc_quant = False
input_quant = True

for noise_level in range(3,4):
    for b in range(2, 11):
        configurations = config.parse_config('../yaml/example.yaml')

        noise_level_db = noise_level
        noise_level_linear = utils.db_to_linear(noise_level_db)

        configurations.signal.num_pos_decision = 1000
        configurations.signal.amps = [1]
        configurations.noise.init_args.top = noise_level_linear
        configurations.noise.init_args.steady_state = noise_level_linear

        generator = signal_generator.InputSignalGenerator(configurations.signal, configurations.noise)
        signal_with_noise, _ = generator.get()

        configurations.noise.init_args.top = 0
        configurations.noise.init_args.steady_state = 0

        generator = signal_generator.InputSignalGenerator(configurations.signal, configurations.noise)
        signal_without_noise, _ = generator.get()

        noise_sample = signal_with_noise - signal_without_noise

        Nd = configurations.signal.num_pos_decision
        N = configurations.signal.block_size
        signal_with_noise = signal_with_noise[0:Nd,:,:]
        signal_without_noise = signal_without_noise[0:Nd,:,:]
        noise_sample = noise_sample[0:Nd,:,:]

        coeff_jdht = np.zeros((Nd,N,N))
        for i in range(Nd):
            coeff_jdht[i, :, :] = freq_transform.dht_coeff_jitter(N)/N

        var_jdht_coeff = np.var(coeff_jdht)
        signal_power = np.var(signal_without_noise)

        Bw = b
        w_max = np.abs(coeff_jdht).max()
        qcoeff_jdht = unisign_quant(coeff_jdht, Bw, w_max, weight_quant)
        deltaw = utils.get_delta(Bw, w_max)

        Bx = b
        sigal_max = np.abs(signal_with_noise).max()
        qsignal_with_noise =  unisign_quant(signal_with_noise, Bx, sigal_max, input_quant)
        deltax = utils.get_delta(Bx, sigal_max)

        result_noise = np.zeros(signal_without_noise.shape)
        result_signal_without_noise = np.zeros(signal_without_noise.shape)
        result_signal_with_noise = np.zeros(signal_without_noise.shape)
        qresult_signal_without_noise = np.zeros(signal_without_noise.shape)
        qresult_signal_with_noise = np.zeros(signal_without_noise.shape)

        start = datetime.now()
        for i in range(Nd):
            result_signal_without_noise[i,:,:] = signal_without_noise[i,:,:].dot(coeff_jdht[i,:,:].T)
            result_noise[i,:,:]  = noise_sample[i,:,:].dot(coeff_jdht[i,:,:].T)
            result_signal_with_noise[i, : ,:] = signal_with_noise[i,:,:].dot(coeff_jdht[i,:,:].T)
            
            qresult_signal_with_noise[i, :, :] = qsignal_with_noise[i,:,:].dot(qcoeff_jdht[i,:,:].T)

        Ba = b
        result_max = np.abs(result_signal_with_noise).max()
        deltay = utils.get_delta(Ba, result_max)
        qresult_signal_with_noise = unisign_quant(qresult_signal_with_noise, Ba, result_max, acc_quant)
        end = datetime.now()

        total_dp_noise = qresult_signal_with_noise - result_signal_without_noise
        var_total_dp_noise = total_dp_noise.var()
        A = deltay**2/12 if acc_quant else 0
        B = N * var_jdht_coeff * noise_level_linear 
        C = N/12 * var_jdht_coeff * deltax**2 if input_quant else 0
        D = N/12 * signal_without_noise.var() * deltaw**2 if weight_quant else 0
        E = N/12 * noise_level_linear * deltaw**2 if weight_quant else 0
        
        predicted_signal_power = N * signal_power * var_jdht_coeff
        sim_signal_power = result_signal_without_noise.var()
        
        print([A,B,C,D,E])
        predicted_snr = predicted_signal_power / (A+B+C+D+E)
        sim_snr = sim_signal_power / var_total_dp_noise
        print('total time (s):', utils.format_float((end-start).total_seconds()), 'B:', b,
              noise_level, 'dB:', 10*np.log10((A+B+C+D+E)/var_total_dp_noise), 
             10*np.log10(predicted_signal_power/sim_signal_power),
             10*np.log10(predicted_snr/sim_snr))

[0, 0.1248993535084581, 0.0555724806133945, 0, 0]
total time (s):  0.007 B: 2 3 dB: 0.04259475041207329 -0.0026474248069216265 -0.04524217521899508
[0, 0.1246366887635359, 0.014943234633885522, 0, 0]
total time (s):  0.009 B: 3 3 dB: 0.05856731741590948 0.03557818650502763 -0.022989130910882852
[0, 0.12494514723587682, 0.0033561550635260386, 0, 0]
total time (s):  0.009 B: 4 3 dB: 0.09803246851525867 -0.030215119244902177 -0.1282475877601606
[0, 0.12504106888716202, 0.0008257297719050891, 0, 0]
total time (s):  0.008 B: 5 3 dB: 0.019488775825314567 -0.014034223143563604 -0.03352299896887766
[0, 0.12446443529852907, 0.00021827013185596816, 0, 0]
total time (s):  0.009 B: 6 3 dB: -0.08434720010123717 -0.034457262190694976 0.04988993791054251
[0, 0.12480599273494539, 5.519345517571308e-05, 0, 0]
total time (s):  0.008 B: 7 3 dB: -0.04870110085954837 0.016893830507234656 0.06559493136678235
[0, 0.12490578422769806, 1.171246953931554e-05, 0, 0]
total time (s):  0.008 B: 8 3 dB: 0.0060630354

In [10]:
nidx = np.where(noise_levels==-20)[0]
sim = simret[nidx, :]
er = evalret[nidx, :]

NameError: name 'noise_levels' is not defined

In [ ]:
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D
plt.figure(figsize=(10,5))
color = [
    '#1f77b4',
    '#ff7f0e',
    '#2ca02c',
    '#d62728',
    '#9467bd',
    '#8c564b',
    '#e377c2',
    '#7f7f7f',
    '#bcbd22',
    '#17becf'
]

legend_elements = [
    Line2D([0], [0], marker='o', color='w', label='Evaluation', markerfacecolor='k', markersize=8),
    Line2D([0], [0], marker='s', color='w', label='Simulation', markerfacecolor='k', markersize=8),
]

for idx, nl in enumerate([-25, -20, -15, -10]):
    nidx = np.where(noise_levels==nl)[0]
    sim = simret[nidx, :]
    er = evalret[nidx, :]
    plt.plot(bs, 10*np.log10(sim[0]), marker='o', c=color[idx])
    plt.plot(bs, 10*np.log10(er[0]), marker='s',c=color[idx])
    legend_elements.append(Line2D([0], [0], color=color[idx], lw=4, label='$SNR_T$='+str(-nl)+' dB'),)

plt.legend(handles=legend_elements, loc='lower right', fontsize=15, ncol=1)
plt.grid()
plt.tick_params('both', labelsize=15)
plt.xlabel('$B_x$', fontsize=15)
plt.ylabel('$SNR_F$' + '(dB)', fontsize=15)

In [ ]:
signal_without_noise

In [ ]:
result_signal_with_noise.shape

In [ ]:
simret[nidx, :]

In [ ]:
configurations.signal